# MMOB Project

# import

in this cell, relevant libraries are imported, as well as the datafile imported from our github and a dict that will store all results during the project 

In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
from biogeme import models
from biogeme import results as res
data_file ='https://raw.githubusercontent.com/DunodMax/MMOB/main/Dataset.txt'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC
database = db.Database('LPMC', LPMC)
all_results = {}

# Model 0

all useful columns for this model are used as variables

In [2]:
travel_mode=Variable('travel_mode')
dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')
pt_interchanges=Variable('pt_interchanges')
dur_driving=Variable('dur_driving')
cost_transit=Variable('cost_transit')
cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
dur_walking=Variable('dur_walking')
dur_cycling=Variable('dur_cycling')

Trip duration for each mode are computed ( see report for details about equations)
Cost for each mode with a cost are computed ( cycling and walking is considered free)

In [3]:
time_walk=dur_walking
time_cycle=dur_cycling
time_pt=dur_pt_access+dur_pt_rail+dur_pt_bus+dur_pt_int
time_drive=dur_driving

cost_drive=cost_driving_ccharge+cost_driving_fuel
cost_pt=cost_transit

There is 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [4]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

utility functions

In [5]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

In [6]:
av = {1: 1, 2: 1, 3: 1, 4:1}

estimation of the parameters for Model 0, all relevant parameters are printed

In [7]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.832163,0.102680,-37.321371,0.0,0.107574,-35.623351,0.0
asc_drive,-1.235707,0.073716,-16.763025,0.0,0.080400,-15.369411,0.0
asc_pt,-0.528767,0.053297,-9.921212,0.0,0.054766,-9.655006,0.0
beta_cost,-0.169780,0.013321,-12.745694,0.0,0.013108,-12.952753,0.0
beta_time,-5.450540,0.181898,-29.964846,0.0,0.202684,-26.891767,0.0


In [8]:
res.compileEstimationResults(all_results)

AttributeError: module 'biogeme.results' has no attribute 'compileEstimationResults'

In [9]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599454

# Model 1 

### Time specification

We will try a specification where the the cost coeffecient is Generic and the time coefficient is alternative specific

Again 4 different travel mode, 4 utility functions and 3 alternative specific constant.
This time we have the generic parameter for cost and a 4 new specific parameter for travel time

In [10]:
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)

New utility functions :

In [11]:
v_walk_model1 = beta_time_walk * time_walk  
v_cycle_model1 = asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1 = asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1 = asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

Estimation of the parameters for Model 1, all relevant parameters are printed

In [12]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model1_time_specification'
all_results['Model1_time_specification'] = biogeme_model1.estimate()
results_alt_spec_time = biogeme_model1.estimate()
results_alt_spec_time.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.509593,0.171871,-26.238171,0.0,0.191028,-23.606920,0.0
asc_drive,-1.885701,0.106697,-17.673479,0.0,0.122740,-15.363361,0.0
asc_pt,-2.339709,0.114622,-20.412388,0.0,0.125798,-18.598938,0.0
beta_cost,-0.146061,0.014772,-9.887595,0.0,0.014850,-9.835754,0.0
beta_time_cycle,-6.121448,0.467854,-13.084093,0.0,0.542680,-11.280025,0.0
beta_time_drive,-6.445066,0.333687,-19.314718,0.0,0.391345,-16.469010,0.0
beta_time_pt,-3.514554,0.233212,-15.070204,0.0,0.258207,-13.611367,0.0
beta_time_walk,-8.204816,0.290737,-28.220736,0.0,0.364365,-22.518121,0.0


In [13]:
res.compileEstimationResults(all_results)

AttributeError: module 'biogeme.results' has no attribute 'compileEstimationResults'

### Cost specification

We compare with the model where we specify the price, to be sure to have the most interesting model for the future models. (walk and cycling don't have cost, so only 2 specific parameter for cost)

In [ ]:
beta_cost_drive = Beta('beta_cost_drive', 0, None, None, 0)
beta_cost_pt = Beta('beta_cost_pt', 0, None, None, 0)

New utility function

In [ ]:
v_walk_model1_spec_cost = beta_time * time_walk  
v_cycle_model1_spec_cost = asc_cycle + beta_time * time_cycle 
v_pt_model1_spec_cost = asc_pt + beta_time * time_pt + beta_cost_pt * cost_pt
v_drive_model1_spec_cost = asc_drive + beta_time * time_drive + beta_cost_drive * cost_drive

In [ ]:
V_model1_spec_cost = {1: v_walk_model1_spec_cost , 2: v_cycle_model1_spec_cost, 3: v_pt_model1_spec_cost, 4: v_drive_model1_spec_cost}
logprob_model1_spec_cost = models.loglogit(V_model1_spec_cost, av, travel_mode)
biogeme_model1_spec_cost = bio.BIOGEME(database, logprob_model1_spec_cost)
biogeme_model1_spec_cost.modelName = 'Model_1_cost_specifiaction'
all_results['Model1_cost_specification'] = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost.getEstimatedParameters()

In [ ]:
res.compileEstimationResults(all_results)

### Comparaison

we compare the Akaike Information Criterion (AIC)

In [14]:
print("Model 0 PIC value :",results_generic.data.akaike)
print("Model 1, time alternative specification PIC value :",results_alt_spec_time.data.akaike)
print("Model 1, cost alternative specification PIC value :",results_alt_spec_cost.data.akaike)

Model 0 PIC value : 9142.89298347718
Model 1, time alternative specification PIC value : 8545.668496833014


NameError: name 'results_alt_spec_cost' is not defined

we compare the Bayesian Information Criterion (BIC)

In [15]:
print("Model 0 PIC value :",results_generic.data.bayesian)
print("Model 1, alternative specification travel time, PIC value :",results_alt_spec_time.data.bayesian)
print("Model 1, alternative specification cost, PIC value :",results_alt_spec_cost.data.bayesian)

Model 0 PIC value : 9175.478949434262
Model 1, alternative specification travel time, PIC value : 8597.806042364344


NameError: name 'results_alt_spec_cost' is not defined

According to the AIC and the BIC, the best model is the one wih the alternative specific travel time coefficient, and the generic travel cost coefficient.

# Model 2

In [16]:
LPMC.columns

Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
       'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time', 'age', 'female', 'driving_license', 'car_ownership',
       'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
       'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'pt_interchanges',
       'dur_driving', 'cost_transit', 'cost_driving_fuel',
       'cost_driving_ccharge', 'driving_traffic_percent'],
      dtype='object')

In [17]:
LPMC.age.drop_duplicates()

0       71
1       64
3       54
4       49
9       65
        ..
612     88
660     86
763     87
852     89
4853    93
Name: age, Length: 89, dtype: int64

In [18]:
LPMC.age.describe()

count    5000.000000
mean       40.167800
std        18.998806
min         5.000000
25%        26.000000
50%        39.000000
75%        53.000000
max        99.000000
Name: age, dtype: float64

In [19]:
biogeme_model_pref = biogeme_model1 #our prefered model using previous cell

In [20]:
#we're adding the attribute distance to our model 
#we're adding the interaction between age and distance
distance=Variable('distance')
age=Variable('age')

In [21]:
#defining associated coefficients
beta_distance= Beta('beta_distance', 0, None, None, 0)
beta_age_distance= Beta('beta_age_distance', 0, None, None, 0)

In [22]:
#the specification, we only consider the alternative distance for walking
#the interaction distance age is considered just for public trasnport
v_walk_model_pref = v_walk_model1 + beta_distance * distance
v_cycle_model_pref = v_cycle_model1
v_pt_model_pref = v_pt_model1 + beta_age_distance * age * distance
v_drive_model_pref = v_drive_model1

In [23]:
#the estimation results
V_model_pref = {1: v_walk_model_pref , 2: v_cycle_model_pref, 3: v_pt_model_pref, 4: v_drive_model_pref}
logprob_model_pref = models.loglogit(V_model_pref, av, travel_mode)
biogeme_model_pref = bio.BIOGEME(database, logprob_model_pref)
biogeme_model_pref.modelName = 'Model_prefered'
all_results['Model_prefered'] = biogeme_model_pref.estimate()
results_model_pref = biogeme_model_pref.estimate()
results_model_pref.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.449497,1.721565e-01,-25.845644,0.000000e+00,1.911024e-01,-23.283316,0.000000
asc_drive,-1.807570,1.081134e-01,-16.719199,0.000000e+00,1.242581e-01,-14.546902,0.000000
asc_pt,-2.345378,1.162453e-01,-20.176115,0.000000e+00,1.285554e-01,-18.244106,0.000000
beta_age_distance,-0.000002,3.032714e-07,-5.608678,2.038777e-08,3.550320e-07,-4.790981,0.000002
beta_cost,-0.147382,1.500983e-02,-9.819056,0.000000e+00,1.525602e-02,-9.660601,0.000000
beta_distance,-0.000759,2.507486e-04,-3.026023,2.477939e-03,2.850129e-04,-2.662234,0.007762
beta_time_cycle,-6.698642,4.795484e-01,-13.968645,0.000000e+00,5.588156e-01,-11.987213,0.000000
beta_time_drive,-7.249854,3.713098e-01,-19.525079,0.000000e+00,4.394175e-01,-16.498784,0.000000
beta_time_pt,-3.153433,2.389975e-01,-13.194422,0.000000e+00,2.657520e-01,-11.866076,0.000000
beta_time_walk,-5.384342,1.017018e+00,-5.294245,1.195094e-07,1.149130e+00,-4.685583,0.000003


In [24]:
res.compileEstimationResults(results_model_pref)

AttributeError: module 'biogeme.results' has no attribute 'compileEstimationResults'

In [27]:
#here I'll do (maybe) BIC, AIK, and loglikehood ratio between model pref(base model) and model_augmented 
#(with the new features)